In [ ]:
%%capture
!pip install captum

Using The Captum Library for Image Salience Mapping
===================================================

Based on https://pytorch.org/tutorials/recipes/recipes/Captum_Recipe.html

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Model Interpretability using Captum
===================================


Captum helps you understand how the data features impact your model
predictions or neuron activations, shedding light on how your model
operates.

Using Captum, you can apply a wide range of state-of-the-art feature
attribution algorithms such as `Guided GradCam` and
`Integrated Gradients` in a unified way.

In this recipe you will learn how to use Captum to:

-   Attribute the predictions of an image classifier to their
    corresponding image features.
-   Visualize the attribution results.


Before you begin
================


Make sure Captum is installed in your active Python environment. Captum
is available both on GitHub, as a `pip` package, or as a `conda`
package. For detailed instructions, consult the installation guide at
<https://captum.ai/>


For a model, we use a built-in image classifier in PyTorch. Captum can
reveal which parts of a sample image support certain predictions made by
the model.


In [ ]:
import torchvision
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

model = torchvision.models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).eval()

response = requests.get("https://image.freepik.com/free-photo/two-beautiful-puppies-cat-dog_58409-6024.jpg")
img = Image.open(BytesIO(response.content))

center_crop = transforms.Compose([
 transforms.Resize(256, max_size=300),
 transforms.CenterCrop(224),
])

normalize = transforms.Compose([
    transforms.ToTensor(),               # converts the image to a tensor with values between 0 and 1
    transforms.Normalize(                # normalize to follow 0-centered imagenet pixel RGB distribution
     mean=[0.485, 0.456, 0.406],
     std=[0.229, 0.224, 0.225]
    )
])
input_img = normalize(center_crop(img)).unsqueeze(0)

Computing Attribution using Occlusion
=====================================


Among the top-3 predictions of the models are classes 208 and 283 which
correspond to dog and cat.

Let us attribute each of these predictions to the corresponding part of
the input, using Captum's `Occlusion` algorithm.


In [ ]:
from captum.attr import Occlusion

occlusion = Occlusion(model)

strides = (3, 9, 9)               # smaller = more fine-grained attribution but slower
target=208,                       # Labrador index in ImageNet
sliding_window_shapes=(3, 45, 45) # choose size enough to change object appearance
baselines = 0                     # values to occlude the image with. 0 corresponds to gray

attribution_dog = occlusion.attribute(
    input_img,
    strides = strides,
    target=target,
    sliding_window_shapes=sliding_window_shapes,
    baselines=baselines)


target=283,                       # Persian cat index in ImageNet
attribution_cat = occlusion.attribute(
    input_img,
    strides = strides,
    target=target,
    sliding_window_shapes=sliding_window_shapes,
    baselines=baselines)

Besides `Occlusion`, Captum features many algorithms such as
`Integrated Gradients`, `Deconvolution`, `GuidedBackprop`,
`Guided GradCam`, `DeepLift`, and `GradientShap`. All of these
algorithms are subclasses of `Attribution` which expects your model as a
callable `forward_func` upon initialization and has an `attribute(...)`
method which returns the attribution result in a unified format.

Let us visualize the computed attribution results in case of images.


Visualizing the Results
=======================


Captum's `visualization` utility provides out-of-the-box methods to
visualize attribution results both for pictorial and for textual inputs.


In [ ]:
import numpy as np
from captum.attr import visualization as viz

def visualize_results(img, attribution, methodname, title):
    # Convert the compute attribution tensor into an image-like numpy array
    attribution_np = np.transpose(attribution[0].cpu().detach().numpy(), (1,2,0))
    vis_types = ["original_image", "heat_map", "blended_heat_map"]
    vis_signs = ["all", "all", "positive"] # "positive", "negative", or "all" to show both
    # positive attribution indicates that the presence of the area increases the prediction score
    # negative attribution indicates distractor areas whose absence increases the score
    _ = viz.visualize_image_attr_multiple(
        attribution_np,
        np.array(center_crop(img)),
        vis_types,
        vis_signs,
        ["image", methodname + " attribution for " + title, "overlay"],
        show_colorbar=True,
        )

visualize_results(img, attribution_dog, "Occlusion", "dog")
visualize_results(img, attribution_cat, "Occlusion", "cat")

Demonstration of Integrated Gradients
-------------------------------------

In [ ]:
from captum.attr import IntegratedGradients

intgrad = IntegratedGradients(model)

target=208,                       # Labrador index in ImageNet
baselines = 0                     # values to occlude the image with. 0 corresponds to gray

ig_attribution_dog = intgrad.attribute(input_img, baselines=baselines, target=target, n_steps=50)


target=283,                       # Persian cat index in ImageNet
ig_attribution_cat = intgrad.attribute(input_img, baselines=baselines, target=target, n_steps=50)

visualize_results(img, ig_attribution_dog, "IntegratedGradients", "dog")
visualize_results(img, ig_attribution_cat, "IntegratedGradients", "cat")

Demonstration of SmoothGrad modification of Integrated Gradients
----------------------------------------------------------------

In [ ]:
from captum.attr import NoiseTunnel

smoothgrad = NoiseTunnel(intgrad)

target=208,                       # Labrador index in ImageNet
baselines = 0                     # values to occlude the image with. 0 corresponds to gray

smoothgrad_attribution_dog = smoothgrad.attribute(
    input_img, nt_type='smoothgrad', nt_samples=10, target=target)


target=283,                       # Persian cat index in ImageNet
smoothgrad_attribution_cat = smoothgrad.attribute(
    input_img,  nt_type='smoothgrad', nt_samples=10, target=target)

visualize_results(img, smoothgrad_attribution_dog, "SmoothGrad", "dog")
visualize_results(img, smoothgrad_attribution_cat, "SmoothGrad", "cat")


Demonstration of GradCAM
------------------------

In [ ]:
from captum.attr import LayerGradCam, LayerAttribution

gradcam = LayerGradCam(model, model.layer4)

target = 208
gradcam_attribution_dog = gradcam.attribute(
    input_img, target=target)
up_gradcam_attribution_dog = LayerAttribution.interpolate(
    gradcam_attribution_dog, input_img.shape[2:])


target=283,                       # Persian cat index in ImageNet
gradcam_attribution_cat = gradcam.attribute(
    input_img, target=target)
up_gradcam_attribution_cat = LayerAttribution.interpolate(
    gradcam_attribution_cat, input_img.shape[2:])

visualize_results(img, up_gradcam_attribution_dog, "GradCAM", "dog")
visualize_results(img, up_gradcam_attribution_cat, "GradCAM", "cat")

In [ ]:
gradcam_attribution_dog.shape

Final Notes
===========


You should play with and customize this notebook.

* Try some other images and target classes.  In particular, we have provided a few interesting images to try:

https://sidn.baulab.info/salience/images/dog-and-cat.jpg
https://sidn.baulab.info/salience/images/rv-and-speedboat.jpg
https://sidn.baulab.info/salience/images/toaster-scene.jpg

* Try reproducing the Adebayo results

Be sure to think about and answer the reading questions from today.